In [1]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt

In [2]:
status_codes = {
    1: 'LOADED',
    2: 'OPTIMAL',
    3: 'INFEASIBLE',
    4: 'INF_OR_UNBD',
    5: 'UNBOUNDED',
    6: 'CUTOFF',
    7: 'ITERATION_LIMIT',
    8: 'NODE_LIMIT',
    9: 'TIME_LIMIT',
    10: 'SOLUTION_LIMIT',
    11: 'INTERRUPTED',
    12: 'NUMERIC',
    13: 'SUBOPTIMAL',
    14: 'INPROGRESS',
    15: 'USER_OBJ_LIMIT'
}

In [3]:
A_3_1 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0]
])

A_3_2 = np.array([
    [1, 2, 0, 0],
    [0, 1, -2, 0]
])

A_3_3 = np.array([
    [1, 3, 3, 0],
    [0, -1, 3, -3]
])

In [69]:
k1, k2 = 5, 1

delta = 0.5

gamma = np.array([
    0, 4, 2, -5
])

lam = k1 / k2
y_true = np.array([
    1,
    lam,
    lam**2 + lam,
    lam**3 + 3*(lam**2) + lam
])

y_true += gamma

y_lb = y_true - (delta / 2)
y_ub = y_true + (delta / 2)

y_lb[0] = 1
y_ub[0] = 1

In [70]:
def M_0_constructor(y):
    M = np.array([
        [y[0], y[1]],
        [y[1], y[2]]
    ])
    return M

def M_1_constructor(y):
    M = np.array([
        [y[1], y[2]],
        [y[2], y[3]]
    ])
    return M

In [71]:
options = {}
options['OutputFlag'] = 0
time_limit = 300

# environment context
with gp.Env(params=options) as env:

    # model context
    with gp.Model('test-SDP', env=env) as model:

        # model settings
        model.Params.TimeLimit = time_limit
        K = 100

        # variables
        y = model.addMVar(shape=4, vtype=GRB.CONTINUOUS, name="y", lb=0)
        k = model.addMVar(shape=2, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)
        M_0 = gp.MVar.fromlist([
            [y[0].item(), y[1].item()],
            [y[1].item(), y[2].item()]
        ])
        M_1 = gp.MVar.fromlist([
            [y[1].item(), y[2].item()],
            [y[2].item(), y[3].item()]
        ])

        # constraints

        # moment bounds
        model.addConstr(y <= y_ub, name="y_UB")
        model.addConstr(y >= y_lb, name="y_LB")

        # moment equations
        model.addConstr(k.T @ A_3_1 @ y == 0, name="ME_1")
        #model.addConstr(k.T @ A_3_2 @ y == 0, name="ME_2")
        #model.addConstr(k.T @ A_3_3 @ y == 0, name="ME_3")

        # base
        model.addConstr(y[0] == 1, name="y0_base")

        # fixed parameter
        model.addConstr(k[1] == k2, name="k2_fixed")

        # optimize
        solution = {}
        model.setObjective(0, GRB.MINIMIZE)
        model.optimize()
        status = status_codes[model.status]
        print(f"Status {status}")

        # found feasible point
        while status == "OPTIMAL":

            # get moment matrix values
            M_0_val = M_0.X
            M_1_val = M_1.X

            # check semidefinite
            evals_0, evecs_0 = np.linalg.eigh(M_0_val)
            evals_1, evecs_1 = np.linalg.eigh(M_1_val)

            print(evals_0)
            print(evals_1)

            # positive eigenvalues
            if (evals_0 >= 0).all() and (evals_1 >= 0).all():
                print("SDP feasible")
                break

            # negative eigenvalue
            else:

                if evals_0[0] < 0:

                    # get eigenvector
                    v = evecs_0[0]

                    # add cutting plane
                    model.addConstr(v.T @ M_0 @ v >= 0, name="Cut_0")

                    print("M_0 cut")

                if evals_1[0] < 0:

                    # get eigenvector
                    v = evecs_1[0]

                    # add cutting plane
                    model.addConstr(v.T @ M_1 @ v >= 0, name="Cut_1")

                    print("M_1 cut")

                # resolve
                solution = {}
                model.setObjective(0, GRB.MINIMIZE)
                model.optimize()
                status = status_codes[model.status]
                print(f"Status {status}")

Status OPTIMAL
[-1.31691664 34.06691664]
[  3.61331008 204.88959168]
M_0 cut
Status INFEASIBLE
